# Series

## Import the Polars Library
- The `import` keyword loads a named construct from either a Python module or from a package.
- The `polars` library is often assigned the alias `pl`.
- The `pl` alias acts as a namespace for the constructs that are nested within Polars.

In [1]:
import polars as pl

- Add a dot after the name, then press the Tab key to see top-level attributes and functions.
- The `__version__` attribute outputs the installed version of Polars.
- The `show_versions` function returns the versions of Polars, Python, and other dependencies alongside system info.

In [2]:
pl.__version__

'1.37.1'

In [3]:
pl.show_versions()

--------Version info---------
Polars:              1.37.1
Index type:          UInt32
Platform:            macOS-26.2-arm64-arm-64bit-Mach-O
Python:              3.13.11 (main, Dec  9 2025, 20:26:22) [Clang 21.1.4 ]
Runtime:             rt32

----Optional dependencies----
Azure CLI            <not installed>
adbc_driver_manager  1.7.0
altair               5.5.0
azure.identity       <not installed>
boto3                <not installed>
cloudpickle          3.1.1
connectorx           0.4.3
deltalake            1.1.4
fastexcel            0.14.0
fsspec               2025.7.0
gevent               25.5.1
google.auth          <not installed>
great_tables         0.18.0
matplotlib           3.10.5
numpy                2.3.2
openpyxl             3.1.5
pandas               3.0.0rc2
polars_cloud         <not installed>
pyarrow              21.0.0
pydantic             2.11.7
pyiceberg            0.9.1
sqlalchemy           2.0.43
torch                <not installed>
xlsx2csv             0.8.4
xlsxwr

### Further Reading
- https://docs.pola.rs/api/python/stable/reference/api/polars.show_versions.html

## Intro to the Series
- A `Series` is a single column of homogenous data. Homogenous means "of the same data type".
- A `Series` is a 1-dimensional data structure.

![Series data structure](images/Series.png)

## Create a Series
- The `pl.Series` constructor at the top of Polars supports a variety of data inputs.
- Press Shift + Tab inside the parentheses to see documentation.
- The `values` parameter supplies the data for the `Series`.

In [4]:
pl.Series(["Doritos", "Cheetos", "Cheez-Its"])

""
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


- If we pass data directly as the first parameter, Polars will assume the `Series` has no name (an anti-pattern).
- The recommended pattern is to pass in a `name` as the first parameter, then the data.

In [5]:
pl.Series("Chips", ["Doritos", "Cheetos", "Cheez-Its"])
pl.Series(name="Chips", values=["Doritos", "Cheetos", "Cheez-Its"])
pl.Series(values=["Doritos", "Cheetos", "Cheez-Its"], name="Chips")
pl.Series("Chips", values=["Doritos", "Cheetos", "Cheez-Its"])

Chips
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


### Further Reading
- https://docs.pola.rs/user-guide/concepts/data-types-and-structures/#series

## Data Type Inference
- Polars will infer the data type of the `Series` from the provided values.
- The `dtype` parameter sets the data type of the `Series` values. Use it to overwrite Polars' default type inference.
- Polars may abbreviate the data type (e.g, `pl.Float64` will be `f64` and `pl.String` will be `str`).

In [6]:
pl.Series("Prices", [1.99, 2.99, 3.49])
pl.Series("Prices", [1.99, 2.99, 3.49], pl.Float64)
pl.Series("Prices", [1.99, 2.99, 3.49], dtype=pl.Float64)

Prices
f64
1.99
2.99
3.49


- `Float64` (64 bits/8 bytes) and `Float32` (32 bits/4 bytes) are both floating-point (decimal) types.
- A `f64` offers greater precision (15-17 digits) than a `f32` (7 digits) but occupies twice the memory.
- Prioritize code simplicity and business value before optimizing for efficiency.

In [7]:
pl.Series("Prices", [1.99, 2.99, 3.49], pl.Float32)

Prices
f32
1.99
2.99
3.49


- Polars will throw an error if the values' data type does not match the `dtype` argument.

In [8]:
# pl.Series("Prices", [1.99, 2.99, 3.49], dtype=pl.String)

- Set the `strict` parameter to `False` to coerce/cast the data types into the specified `dtype`.

In [9]:
pl.Series("Prices", [1.99, 2.99, 3.49], dtype=pl.String, strict=False)
pl.Series("Prices", [1.99, 2.99, 3.49, "4.49"], dtype=pl.String, strict=False)
pl.Series("Prices", [1.99, 2.99, 3.49, "4.49"], dtype=pl.Float64, strict=False)
pl.Series("Prices", [1.99, 2.99, 3.49, "nonsense"], dtype=pl.Float64, strict=False)

Prices
f64
1.99
2.99
3.49
null


### Further Reading
- https://docs.pola.rs/api/python/stable/reference/series/index.html

## Attributes
- An **attribute** is a piece of information about an object.
- To access an attribute value, write the object, then a dot and the attribute name.
- The `name` attribute provides the name of the `Series`.
- The `dtype` attribute provides the data type of the `Series`.
- The `shape` attribute provides a tuple with the dimensions (width x height) of a Polars data structure.

In [10]:
chips = pl.Series("Chips", ["Doritos", "Cheetos", "Cheez-Its"])
chips

Chips
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


In [11]:
chips.name

'Chips'

In [12]:
chips.dtype

String

In [13]:
chips.shape

(3,)

### Further Reading
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.name.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.dtype.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.shape.html

## Missing Values

- Polars marks missing entries in a `Series` with a `null` value.
- Python's `None` type represents the absence of a value.
- Polars will convert a `None` in a Python list to a `null`/missing value.

In [14]:
temps = pl.Series("Temperatures", [0.0, 85.3, 87.3, None, 92.4])
temps

Temperatures
f64
0.0
85.3
87.3
null
92.4


- Polars supports mathematical operations on `Series`.
- We can add, subtract, multiply, divide, etc every `Series` value with a consistent value.
- For every mathematical operation, Polars returns a new `Series` storing the new values.
- Most operations on `null` values produce `null` values.
- The `+` symbol adds a scalar value to every row value.

In [15]:
temps + 5.5

Temperatures
f64
5.5
90.8
92.8
null
97.9


- Polars will return a `NaN` (not a number) value for invalid numeric operations.
- One example of an invalid operation is dividing 0 by 0.
- `NaN` is a different value from `null`.

In [16]:
temps / 0

Temperatures
f64
NaN
inf
inf
null
inf


- Polars considers `NaN` to be distinctly separate from `null`.
- A `NaN` is a value of a float type (`f64`).

### Further Reading
- https://docs.pola.rs/user-guide/expressions/missing-data/#null-and-nan-values
- https://docs.pola.rs/user-guide/expressions/missing-data/#not-a-number-or-nan-values

## The alias Method

- A **method** is a function attached an object.
- A **method** is a command that we issue to an object. It asks the object to performs an action.
- Write the object, a dot, the method name, and a pair of parentheses.
- Like functions, methods can accept inputs (arguments) and produce an output (return value).
- The `alias` method returns a new `Series` with a new name.

In [17]:
chips = pl.Series("Chips", ["Doritos", "Cheetos", "Cheez-Its"])
chips

Chips
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


In [18]:
chips.alias("Snacks")

Snacks
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


- The `alias` method returns a copy of the `Series`. The original `Series` is unaffected.
- Reassign a method's return value to a variable to overwrite the variable's original value.
- Polars methods will usually return a new object/copy. 

In [19]:
chips = chips.alias("Snacks")

In [20]:
chips

Snacks
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


### Further Reading
- https://docs.pola.rs/user-guide/expressions/expression-expansion/#renaming-a-single-column-with-alias
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.alias.html

## Import a CSV File

- The comma-separated values (CSV) format stores one record per row.
- The CSV format separates each row's values with commas.

```
name,strength,magic,speed
Zorblax,9,2,5
Mira,4,9,8
Thud,10,1,2
Eloria,3,10,7
```

- The `read_csv` function imports a comma-separated values (CSV) file.
- The `read_csv` function returns a `DataFrame`, a table that holds 1 or more `Series`.
- Notice the `shape` output includes the number of rows and the number of columns.
- The underscores in `1_000` are for readability.

In [21]:
ice_creams = pl.read_csv("ice_creams.csv")
ice_creams

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
…
"""Mango Fudge Rush"""
"""Honey Fudge Surprise"""
"""Cinnamon Cookie Bliss"""


In [22]:
type(ice_creams)

polars.dataframe.frame.DataFrame

- The `to_series` method converts a `DataFrame` column into a `Series`.
- With a 1-column `DataFrame`, Polars assumes it should use the only available column.
- If a `DataFrame` has multiple columns, we have to provide the column's numeric index.
- Polars assigns the columns an index (position in line) starting at 0.

In [23]:
ice_creams.to_series()

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
…
"""Mango Fudge Rush"""
"""Honey Fudge Surprise"""
"""Cinnamon Cookie Bliss"""


- Method chaining invokes a method on the return value of a previous method invocation.

In [24]:
ice_creams = pl.read_csv("ice_creams.csv").to_series()
ice_creams

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
…
"""Mango Fudge Rush"""
"""Honey Fudge Surprise"""
"""Cinnamon Cookie Bliss"""


### Further Reading
- https://docs.pola.rs/user-guide/io/csv/#read-write
- https://docs.pola.rs/api/python/stable/reference/api/polars.read_csv.html

## The head and tail Methods
- The `head` method returns rows from the top/beginning of the `Series`.
- The `tail` method returns rows from the bottom/end of the `Series`.
- By default, both methods return 10 rows. You can pass in a custom number of rows to return.

In [25]:
ice_creams = pl.read_csv("ice_creams.csv").to_series()
ice_creams

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
…
"""Mango Fudge Rush"""
"""Honey Fudge Surprise"""
"""Cinnamon Cookie Bliss"""


In [26]:
ice_creams.head()

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
"""Pistachio Butter Crave"""
"""Espresso Brownie Rush"""
"""Peach Flake Storm"""
"""Coconut Flake Delight"""


In [27]:
ice_creams.head(3)
ice_creams.head(n=7)

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
"""Pistachio Butter Crave"""
"""Espresso Brownie Rush"""


- The `limit` method is an alias for the `head` method.

In [28]:
ice_creams.limit(7)

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
"""Pistachio Butter Crave"""
"""Espresso Brownie Rush"""


In [29]:
ice_creams.tail(3)
ice_creams.tail(n=4)

ice_cream_flavor
str
"""Honey Fudge Surprise"""
"""Cinnamon Cookie Bliss"""
"""Coconut Fudge Twist"""
"""Cinnamon Crumb Magic"""


- A negative value returns all rows from the top _except for_ the specified number of rows to exclude from the bottom.
- For example, `head(-3)` returns all rows except for the last 3.

In [30]:
ice_creams.head(-3)

ice_cream_flavor
str
"""Cherry Chip Drizzle"""
"""Mango Marshmallow Magic"""
"""Vanilla Sprinkle Storm"""
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
…
"""Vanilla Swirl Delight"""
"""Chocolate Graham Blizzard"""
"""Honey Caramel Delight"""


- With a negative value, the `tail` method returns all rows from the bottom except the specified number of rows to exclude from the top.
- For example, `tail(-3)` returns all rows except for the top 3.

In [31]:
ice_creams.tail(-3)

ice_cream_flavor
str
"""Blueberry Flake Delight"""
"""Vanilla Chunk Crave"""
"""Pistachio Butter Crave"""
"""Espresso Brownie Rush"""
"""Peach Flake Storm"""
…
"""Mango Fudge Rush"""
"""Honey Fudge Surprise"""
"""Cinnamon Cookie Bliss"""


- The `first` and `last` methods return the first and last `Series` value.

In [32]:
ice_creams.first()

'Cherry Chip Drizzle'

In [33]:
ice_creams.last()

'Cinnamon Crumb Magic'

### Further Reading
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.head.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.tail.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.limit.html

## Memory Optimization and the schema_overrides Parameter
- Polars infers a column's data type from its values. It defaults to an `i64` (64-bit integer) for whole numbers.

In [34]:
daily_steps = pl.read_csv("daily_steps.csv").to_series()
daily_steps.head(5)

daily_steps
i64
6030
12448
13482
8486
6381


- The `estimated_size` method returns the memory size of the `Series` in bytes.
- Most computers assign 8 bits to 1 byte. 64 bits for one integer is equal to 8 bytes.
- The `Series` has 1000 rows of integers.
- 1000 integers * 8 bytes per integer = 8000 bytes.
- An `i64` supports values from -9,223,372,036,854,775,808 to 9,223,372,036,854,775,807 (-9 quintillion to 9 quintillion).

In [35]:
daily_steps.estimated_size()

8000

In [36]:
daily_steps.min()
daily_steps.max()

24930

- The `schema_overrides` parameter accepts a dictionary to overwrite the inferred type.
- The `schema` parameter defines the mapping for every column in the data structure.
- For a 1-column `DataFrame`, the `schema_overrides` and `schema` parameters are equivalent.
- Add a key-value for each column to override. Use the column name for the key and the desired type for its value.

In [37]:
daily_steps = pl.read_csv(
    "daily_steps.csv", schema_overrides={"daily_steps": pl.Int16}
).to_series()
daily_steps.head(5)

daily_steps
i16
6030
12448
13482
8486
6381


- An `Int16` supports values from -32,768 to 32,767.
- If the dataset has a value greater than 32,767, the import will trigger an error.
- The `Series` still has 1000 rows, but each integer occupies 16 bits (2 bytes).
- 1000 integers * 2 bytes per integer = 2000 bytes (a quarter of the original).

In [38]:
daily_steps.estimated_size()

2000

- We can't take negative steps in a day, so an unsigned integer (0 or positive) type feels ideal.
- Unsigned integers do not support negative values but extend twice as long in the positive direction.
- An `UInt16` supports values from 0 to 65,535.

In [39]:
daily_steps = pl.read_csv(
    "daily_steps.csv", schema_overrides={"daily_steps": pl.UInt16}
).to_series()
daily_steps.head(5)

daily_steps
u16
6030
12448
13482
8486
6381


- The size remains the same (1000 integers x 2 bytes per integer).

In [40]:
daily_steps.estimated_size()

2000

## Sorting a Series
- The `sort` method sorts the values of a `Series`.
- An ascending sort sorts from smallest to greatest (alphabetically for strings).
- A descending sort sorts from greatest to smallest (reverse alphabetically for strings).
- The `descending` parameter sets the sort order. Its default argument is `False` (an ascending sort).

In [41]:
ice_creams = pl.read_csv("ice_creams.csv").to_series()

daily_steps = pl.read_csv(
    "daily_steps.csv", schema_overrides={"daily_steps": pl.UInt16}
).to_series()

In [42]:
daily_steps.sort()
daily_steps.sort(descending=False)
daily_steps.sort(descending=True)

daily_steps
u16
24930
24920
24870
24804
24770
…
5037
5028
5022


In [43]:
ice_creams.sort()
ice_creams.sort(descending=False)
ice_creams.sort(descending=True)

ice_cream_flavor
str
"""Vanilla Toffee Drizzle"""
"""Vanilla Toffee Crave"""
"""Vanilla Swirl Delight"""
"""Vanilla Swirl Bliss"""
"""Vanilla Sprinkle Storm"""
…
"""Almond Brownie Wave"""
"""Almond Brownie Storm"""
"""Almond Bits Wave"""


- Polars sorts capital letters before lowercase ones.

In [44]:
pl.Series(name="text", values=["B", "A", "b"]).sort()

text
str
"""A"""
"""B"""
"""b"""


- The `sort` method returns a new sorted `Series`. The original `Series` is not mutated.
- The `sort` method's `in_place` parameter mutates the original `Series`.
- This parameter is relatively uncommon; Polars usually returns a copy of the data structure.

In [45]:
ice_creams.sort(in_place=True)

ice_cream_flavor
str
"""Almond Bits Charm"""
"""Almond Bits Surprise"""
"""Almond Bits Wave"""
"""Almond Brownie Storm"""
"""Almond Brownie Wave"""
…
"""Vanilla Sprinkle Storm"""
"""Vanilla Swirl Bliss"""
"""Vanilla Swirl Delight"""


In [46]:
ice_creams

ice_cream_flavor
str
"""Almond Bits Charm"""
"""Almond Bits Surprise"""
"""Almond Bits Wave"""
"""Almond Brownie Storm"""
"""Almond Brownie Wave"""
…
"""Vanilla Sprinkle Storm"""
"""Vanilla Swirl Bliss"""
"""Vanilla Swirl Delight"""


### Further Reading
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.sort.html

## Mathematical Methods

- The `len` method returns the length of the `Series` (the number of rows). Polars will include `null` values.
- The `count` method counts the number of non-null values.

In [47]:
sugars = pl.read_csv("sugars.csv").to_series()
sugars

Sugars
f64
3.456
8.0
6.872
5.513
null
8.0


In [48]:
sugars.len()

6

In [49]:
sugars.count()

5

- The `null_count` method counts the number of null/missing values in the `Series`.
- Polars will exclude `NaN` values from the count in the `null_count` method.

In [50]:
sugars.null_count()

1

- A `Series` supports many common statistical operations.
- The `sum` method adds together the `Series` values.
- The `mean` method calculates the average of the `Series` values.
- The `product` method multiplies together the`Series` values.
- The `max` method identifies the greatest value in the `Series`.
- The `min` method identifies the smallest value in the `Series`.
- The `median` method identifies the midpoint/middle value of a sorted `Series`.

In [51]:
sugars.sum()

31.841

In [52]:
sugars.mean()

6.3682

In [53]:
sugars.product()

8379.630157824

In [54]:
sugars.max()

8.0

In [55]:
sugars.min()

3.456

In [56]:
sugars.median()

6.872

- The `mode` method returns a `Series` with the most occuring value(s). It will contain one or more values.
- The `std` method returns the standard deviation of the `Series` (the deviation from the mean).

In [57]:
sugars.mode()

Sugars
f64
8.0


In [58]:
sugars.std()

1.9229945917760662

- The `describe` method returns a `DataFrame` with various statistics.

In [59]:
sugars.describe()

statistic,value
str,f64
"""count""",5.0
"""null_count""",1.0
"""mean""",6.3682
"""std""",1.922995
"""min""",3.456
"""25%""",5.513
"""50%""",6.872
"""75%""",8.0
"""max""",8.0


### Further Reading
- https://docs.pola.rs/user-guide/expressions/missing-data/#missing-data-metadata
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.len.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.count.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.null_count.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.sum.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.mode.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.median.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.product.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.std.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.describe.html

## Rounding Methods
- The `ceil` method rounds up to the next whole number.
- The `floor` method rounds down to the next whole number.
- The `round` method rounds to the closest whole number.
- Polars rounds values >= .5 up and values < .5 down.

In [60]:
sugars = pl.read_csv("sugars.csv").to_series()
sugars

Sugars
f64
3.456
8.0
6.872
5.513
null
8.0


In [61]:
sugars.ceil()

Sugars
f64
4.0
8.0
7.0
6.0
null
8.0


In [62]:
sugars.floor()

Sugars
f64
3.0
8.0
6.0
5.0
null
8.0


In [63]:
sugars.round()

Sugars
f64
3.0
8.0
7.0
6.0
null
8.0


- Precision is the number of digits after a decimal point.
- Pass the `decimals` parameter the precision to round to.
- For example, `3.456` with a precision of 2 will round to `3.46`.

In [64]:
sugars.round(2)

Sugars
f64
3.46
8.0
6.87
5.51
null
8.0


### Further Reading
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.ceil.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.floor.html
- https://docs.pola.rs/api/python/stable/reference/series/api/polars.Series.round.html

## How Polars Differs from Pandas

- A Polars `Series` does not have an index. A Pandas `Series` includes a numeric ascending index starting at 0.
- A Polars `Series` does not support `iloc` and `loc` accessors for retrieving row data by index location or index label.
- A Polars `Series` wraps strings in quotes. A Pandas `Series` does not wrap its strings in quotes.
- A Polars `Series` applies consistent visual styles to its data structures. A Pandas `Series` has no visual formatting.

In [65]:
import pandas as pd

In [66]:
pd_series = pd.Series(data=["Doritos", "Cheetos", "Cheez-Its"], name="Chips")
pd_series

0      Doritos
1      Cheetos
2    Cheez-Its
Name: Chips, dtype: str

In [67]:
pl_series = pl.Series(name="Chips", values=["Doritos", "Cheetos", "Cheez-Its"])
pl_series

Chips
str
"""Doritos"""
"""Cheetos"""
"""Cheez-Its"""


- The Polars `Series` is a different object from the Pandas `Series`.
- Python's `type` function returns the class that a value is made from.

In [68]:
type(pd_series)

pandas.Series

In [69]:
type(pl_series)

polars.series.series.Series

- If an integer column has missing values, Pandas will store the values as floating-points.
- In comparison, Polars retains the original type of the column's values.
- The pandas `Series` constructor includes a `data` parameter. The Polars' equivalent is `values`.
- Pandas uses `NaN` for missing values, Polars uses `null`.

In [70]:
pd.Series(name="Values", data=[100, None, 300])

0    100.0
1      NaN
2    300.0
Name: Values, dtype: float64

In [71]:
pl.Series(name="Values", values=[100, None, 300])

Values
i64
100
null
300


- Pandas will coerce values into a consistent data type if possible.
- In the next example, Pandas coerces the integers 100 and 300 into floating-point values.
- Polars will raise an error if it sees inconsistency in data types (e.g., a mix integer and floating-points).
- Use `strict=False` to skip the safety checks, then coerce the Polars `Series` data into its desired type.

In [72]:
pd.Series(name="Values", data=[100, None, 300.0])

0    100.0
1      NaN
2    300.0
Name: Values, dtype: float64

In [73]:
pl.Series(name="Values", values=[100, None, 300.0], strict=False)

Values
f64
100.0
null
300.0


### Further Reading
- https://docs.pola.rs/user-guide/migration/pandas/#differences-in-concepts-between-polars-and-pandas
- https://docs.pola.rs/user-guide/misc/comparison/#pandas
- https://docs.pola.rs/user-guide/concepts/data-types-and-structures/#appendix-full-data-types-table
- https://pandas.pydata.org/docs/